In [1]:
#import the needed libraries
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
import plotly.graph_objects as go

In [3]:
#import the dataset
file="Service Clients Darty 2017-2021.xlsx"
df = pd.read_excel(file)

#delete the useless columns for this study
#df=df.drop(columns=["Unnamed: 4",'Unnamed: 3',"Appels EGP"])

#from the 1/10/21 the column "appels" is empty, drop these values
df.dropna(inplace=True)

#check the type of the columns "appels"
df.dtypes

#split the data
train_df,test_df=df[:1095],df[1095:]
train_df,test_df=train_df.copy(),test_df.copy()

#set the date as index
train_df.set_index('Date',inplace=True)
test_df.set_index('Date',inplace=True)

# Useful tools

### Resample the dataframe

In [4]:
def resample_df(df, freq):
    """
    Resample a dataframe to a given frequency
    args: 
    - df: dataframe to resample
    - freq: string that gives the frequency ie. 'Y','W','M'
    
    returns: resampled dataframe
    """
    df=df.resample(freq,convention="start").sum()
    if freq=="W" or freq=="w":
        df=df[1:-1]
    return(df)

### Resize the dataframe by year (1 row=1 year)

In [5]:
def resize_yearly(df,resampling='D'):

    #resample the df
    sample_df=resample_df(df, resampling)

    #dealing the Week exception:
    if resampling=="W" or resampling=="w":
        sample_df=sample_df[1:-1]

    #group the sample per year
    group_year=sample_df.groupby(sample_df.index.year)

    #create a dictionnary where key=period and value=data from that period
    params = {}
    params2={}
    for name, group in group_year:
            params[f"{name}"] = group.Appels
            params2[f"{name}"] = group.reset_index().Appels
    return(params,params2)


### Resize the dataframe by month or week 
(this second computation is not need if period=year)

In [6]:
def resize_again(dico,periodicity,year):
    df=dico[year]

    if periodicity=="M" or periodicity=="m":
        groupage=df.groupby(df.index.month)
            
    if periodicity=="W" or periodicity=="w":
        groupage=df.groupby(df.index.week)
        
        #create a dictionnary where key=period and value=data from that period
    params2 = {}
    for name, group in groupage:
        params2[f"{name}"] = group.reset_index().Appels
    
    return params2


### Aggregate the df

In [7]:
def aggregation(df,season,period,stage=0):
    if stage==1:
        df1=resize_yearly(df,season)[0]
        list=[]
        for year in ["2017","2018","2019"]:
            df_period=resize_again(df1,period,year)
            df_period=pd.DataFrame.from_dict(df_period).T
            list.append(df_period)
        tableau=pd.concat(list)
    else:
        df1=resize_yearly(df,season)[1]
        tableau=pd.DataFrame.from_dict(df1).T

        
    return tableau

### Compute the Buys-Ballot table (mean, standard deviation...)

In [8]:
def compile_bbt(df):

    #compute the columns and rows

    #comptute the season_mean column
    season_mean=df.mean(axis=0)
    #comptute the period_mean row
    period_mean=df.mean(axis=1)

    #comptute the season_std column
    season_std=df.std(axis=0)
    #comptute the period_std row
    period_std=df.std(axis=1)

    #create the new columns and rows
    df['period_mean']=period_mean
    df['period_std']=period_std
    df_final = pd.DataFrame({"Season_mean":season_mean,
                    "Season_std":season_std})
    df=df.append(df_final.T)

    return df


### Plot the graph

In [9]:
def plot_decomposition_type(mean_df,std_df,data_period):
    """
    Relationship between the seasonal means and the seasonal standard deviations
    args: 
    - mean_df: dataframe with means values
    - std_df: dataframe with std values
    
    returns: plot of the seasonal means and the pooled standard deviations
    """

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=mean_df.index, y=mean_df,name="Seasonal mean",line = dict(color='black', width=2)))
    fig.add_trace(go.Scatter(x=mean_df.index, y=std_df,name="Seasonal standard deviation",line = dict(color='black', width=1, dash='dash')))
    fig.update_layout(title='Relationship between the seasonal means and the seasonal pooled standard deviations',
                    xaxis_title=data_period,
                    yaxis_title='NUmber of calls')

    return(fig)

In [10]:

##### EN CONSTRUCTION###########
def all_in_one(df_resampled,periodicity,situation):

    if periodicity=="M" or periodicity=="m":
        groupie=df_resampled.groupby(df_resampled.index.month)
            
    elif periodicity=="W" or periodicity=="w":
        groupie=df_resampled.groupby(df_resampled.index.week)

    elif periodicity=="Y":
        groupie=df_resampled.groupby(df_resampled.index.year)
    else:
        return "stop"
    params = {}
    params2={}
    for name, group in groupie:
        if situation ==1:
            params[f"{name}"] = group.Appels
        else:
            params2[f"{name}"] = group.reset_index().Appels
    
    return params,params2


---

# TEST

## Period=Year

In [11]:
#bbt period=année & season=mois
Month_per_year=aggregation(train_df,'M','Y')
bbt_Month_per_year=compile_bbt(Month_per_year)
plot_decomposition_type(bbt_Month_per_year.loc["Season_mean"][:-2],bbt_Month_per_year.loc["Season_std"][:-2],"Months")

#bbt period=année & season=week
Week_per_year=aggregation(train_df,'W','Y')
bbt_Week_per_year=compile_bbt(Week_per_year)
plot_decomposition_type(bbt_Week_per_year.loc["Season_mean"][:-2],bbt_Week_per_year.loc["Season_std"][:-2],"Weeks")

#bbt period=année & season=day
Day_per_year=aggregation(train_df,'D','Y')
bbt_Day_per_year=compile_bbt(Day_per_year)
plot_decomposition_type(bbt_Day_per_year.loc["Season_mean"][:-2],bbt_Day_per_year.loc["Season_std"][:-2],"Days")

/tmp/ipykernel_6403/1257802503.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(df_final.T)
/tmp/ipykernel_6403/1257802503.py:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/tmp/ipykernel_6403/1257802503.py:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



## Period=month


In [58]:
#bbt period=mois & season=week
Week_per_month=aggregation(train_df,'W','M',stage=1)
bbt_Week_per_month=compile_bbt(Week_per_month)
plot_decomposition_type(bbt_Week_per_month.loc["Season_mean"][:-2],bbt_Week_per_month.loc["Season_std"][:-2],"Weeks")
#bbt period=mois & season=day
Day_per_month=aggregation(train_df,'D','M',stage=1)
bbt_Day_per_month=compile_bbt(Day_per_month)
plot_decomposition_type(bbt_Day_per_month.loc["Season_mean"][:-2],bbt_Day_per_month.loc["Season_std"][:-2],"Days")


## Period=week

In [14]:
#bbt period=week & season=day
Day_per_week=aggregation(train_df,'D','W',stage=1)
bbt_Day_per_week=compile_bbt(Day_per_week)
plot_decomposition_type(bbt_Day_per_week.loc["Season_mean"][:-2],bbt_Day_per_week.loc["Season_std"][:-2],"Days")

/tmp/ipykernel_6403/2790320431.py:8: FutureWarning:

weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series. To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)

/tmp/ipykernel_6403/2790320431.py:8: FutureWarning:

weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series. To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)

/tmp/ipykernel_6403/2790320431.py:8: FutureWarning:

weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series. To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)

/tmp/ipykernel_6403/1257802503.py:20: FutureWarning:

The frame.append method is deprecated an

In [18]:
def sorted_bbt(df):
    """TO BE REFACTORED 
    Create the sorted bbt and store in a df 
    """
    list_bbt = []
    for i in range(len(df)):
        list_bbt.append(df.iloc[i].values)

    sort_index = np.argsort(list_bbt)
    sorted_bbt = pd.DataFrame()
    for i in range(len(sort_index)):
        sorted_bbt[i] = sort_index[i]
    return sorted_bbt.T
    


In [20]:
sorted_bbt(Month_per_year.drop(columns=["period_mean","period_std"]))

,0,1,2,3,4,5,6,7,8,9,10,11
0,3,4,1,2,5,0,9,8,7,10,6,11
1,4,5,3,8,9,7,2,1,6,11,0,10
2,3,2,4,1,5,8,9,10,7,0,6,11
